# Libraries

In [1]:
import numpy as np
import math
import sys
import os

import torchvision.transforms as T
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from IPython.display import clear_output

import matplotlib.pyplot as plt
from PIL import Image

from pathlib import Path

# Configurating the model and training parameters

In [2]:
n_epochs = 1000       # number of epochs of training
batch_size = 64       # size of the batches
lr = 0.00005          # learning rate 
n_cpu = 8             # number of cpu threads to use during batch generation
latent_dim = 100      # dimensionality of the latent space
img_size = 64         # size of each image dimension
channels = 3          # number of image channels
n_critic = 5          # number of training steps for discriminator per iter
clip_value = 0.01     # lower and upper clip value for disc. weights
sample_interval = 100 # interval betwen image samples

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

# Prepare data

In [3]:
!gdown --id 1kOP1rMIiaU7nPhTJ4M8EsKIwMcvbz1-N
!unzip houses.zip
clear_output()

In [4]:
class HousesDatasetGAN(Dataset):
    '''our custom version of torch dataset for the pictures of houses'''

    def __init__(self, dataset_dir, transforms=None):
        '''save the paths to images of houses and transforms if needed'''
        self.houses_paths = list(map(str, Path(dataset_dir).rglob('*.jpg')))
        self.transforms   = transforms

    def __len__(self):
        '''size of list of paths'''
        return len(self.houses_paths)

    def __getitem__(self, idx):
        '''retrieve an image from path and return it's tranformed version'''

        # open the image in 3 channels 
        image = Image.open(self.houses_paths[idx]).convert('RGB')

        # transform it if needed
        if self.transforms is not None:
            image = self.transforms(image)

        # return in image and indicator that it is a real one
        return image, 1

In [5]:
# preparing the augmentations 
transforms = T.Compose([
    T.Resize((img_size, img_size)),
    # T.RandomRotation(degrees=45),       #  <-- maybe it's a bad idea
    # T.GaussianBlur(kernel_size=(3, 3)), #  <-- same thing
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# creating a train dataset
houses_dataset = HousesDatasetGAN('validated', transforms)

# Models

## Generator

In [6]:
class GeneratorBig(nn.Module):
    def __init__(self):
        super(GeneratorBig, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 512),
            *block(512, 2048),
            *block(2048, 6144),
            nn.Linear(6144, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

## Discriminator

In [8]:
class DiscriminatorBig(nn.Module):
    def __init__(self):
        super(DiscriminatorBig, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 6144),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(6144, 3072),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(3072, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

# Training

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Initialize generator and discriminator
pretrained = True
if pretrained:
    generator = torch.load('drive/MyDrive/g_big_last.pt')
    discriminator = torch.load('drive/MyDrive/d_big_last.pt')
else:
    generator = GeneratorBig()
    discriminator = DiscriminatorBig()    

if cuda:
    generator.cuda()
    discriminator.cuda()

# Configure data loader
dataloader = torch.utils.data.DataLoader(
    houses_dataset,
    batch_size=batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=lr)

# Tensor type for simplicity
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# Directory for sampels
try:
    os.mkdir('images')
except FileExistsError:
    pass

In [ ]:
batches_done = 30000
for epoch in range(n_epochs):

    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))

            loss_G.backward()
            optimizer_G.step()

        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        batches_done += 1

    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
    )

[Epoch 0/1000] [Batch 0/16] [D loss: -10909.154297] [G loss: 2547.316406]
[Epoch 1/1000] [Batch 0/16] [D loss: -9910.179688] [G loss: 5329.849121]
[Epoch 2/1000] [Batch 0/16] [D loss: -10330.520508] [G loss: 4062.346436]
[Epoch 3/1000] [Batch 0/16] [D loss: -11406.085938] [G loss: -421.159393]
[Epoch 4/1000] [Batch 0/16] [D loss: -10672.092773] [G loss: 6739.904785]
[Epoch 5/1000] [Batch 0/16] [D loss: -10816.071289] [G loss: -43.364407]
[Epoch 6/1000] [Batch 0/16] [D loss: -12087.554688] [G loss: 4306.656738]
[Epoch 7/1000] [Batch 0/16] [D loss: -11218.928711] [G loss: 2177.204834]
[Epoch 8/1000] [Batch 0/16] [D loss: -10398.196289] [G loss: 3785.508545]
[Epoch 9/1000] [Batch 0/16] [D loss: -10905.670898] [G loss: 985.994141]
[Epoch 10/1000] [Batch 0/16] [D loss: -11403.240234] [G loss: -5673.678223]
[Epoch 11/1000] [Batch 0/16] [D loss: -12188.446289] [G loss: 1856.216187]
[Epoch 12/1000] [Batch 0/16] [D loss: -11801.355469] [G loss: -2030.354858]
[Epoch 13/1000] [Batch 0/16] [D loss

In [22]:
torch.save(discriminator, 'drive/MyDrive/d_big_last.pt')
torch.save(generator, 'drive/MyDrive/g_big_last.pt')

In [24]:
z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
gen_imgs = generator(z)
save_image(gen_imgs.data[-25:], "images/lastN.png", nrow=5, normalize=True)

# References

* https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py